In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/598-DLH/Draft
%ls

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/14VE6jxu6gzBuYO9WY79TIloM87C-NZXP/598-DLH/Draft
'Check features in chartevents.ipynb'        store_data_in_tensor_5-7-22.ipynb
 CS598DL4H_project_template_2022Spring.pdf   store_data_in_tensor_5-8-22.ipynb
 DL4H_team_149_Draft.pdf                    'store data in tensor.ipynb'
'old code.gdoc'                             'store notes in tensor.ipynb'
'Preprocessing Notebook.ipynb'               tensor_exports/
 preprocess_text/                            training.ipynb
'Project Draft instructions.gdoc'           'Useful links.gdoc'


In [ ]:
import torch
import pandas as pd
from datetime import datetime
from datetime import timedelta
from math import ceil
import gzip

In [ ]:
!pip3 install accelerate
from accelerate import Accelerator

In [ ]:
%cd tensor_exports

/content/drive/.shortcut-targets-by-id/14VE6jxu6gzBuYO9WY79TIloM87C-NZXP/598-DLH/Draft/tensor_exports


In [ ]:
final_tensor = torch.load('data_tensor_final.pt')

In [ ]:
print(final_tensor.shape)

torch.Size([1541, 5, 720, 826])


In [ ]:
FEATURE_SIZE = int(final_tensor.shape[3])
BATCH_SIZE=100
LEARNING_RATE=1e-2

# Try to get the hadm_idx_to_hadm_id first
so that we can get the is_sepsis label for that hadm_id

In [ ]:
'''
%cd ../mimic
%ls
'''

'\n%cd ../mimic\n%ls\n'

In [ ]:
final_icustays_details = pd.read_pickle('final_icustays_details.pkl')
def get_idx_to_hadm_index_dict(values_pdSeries):
    return {i:v for i, v in enumerate(list(values_pdSeries.unique()))}
    
hadm_idx_id_dict = get_idx_to_hadm_index_dict(final_icustays_details['hadm_id'])
hadm_sepsis = pd.read_pickle('hadm_sepsis.pkl')

In [ ]:
hadm_id_isSepsis_dict = dict(zip(hadm_sepsis.hadm_id, hadm_sepsis.is_sepsis))

# DataLoadder

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data_tensor, hadm_id_isSepsis_dict, hadm_idx_id_dict):
        self.data_tensor = data_tensor
        self.hadm_id_isSepsis_dict = hadm_id_isSepsis_dict
        self.hadm_idx_id_dict = hadm_idx_id_dict
        self._get_available_icu_stays()
        return

    def __len__(self):
        return len(self.available_icu_stays)

    def _get_available_icu_stays(self): ## ICU stays that are not padded
        self.available_icu_stays = []
        for hadm_idx in range(self.data_tensor.shape[0]):
            for icu_idx in range(self.data_tensor.shape[1]):
                if self.data_tensor[hadm_idx, icu_idx, 0, 2] != 0: ## If the first ICU stay's length of stay (los) is zero, then it's padded
                    self.available_icu_stays.append((hadm_idx, icu_idx))

    def _get_sofa_score(self, index, value):
        if index == 57: ## Platelets
            if value >= 150:
                return 0
            elif value >= 100:
                return 1
            else:
                return 2
        elif index == 34: ## creatinie  # => seems to over estimate
            if value < 1.2:
                return 0
            elif value < 2:
                return 1
            else:
                return 2
        elif index == 20: ## FiO2
            return 2 if value >=53.3 else 0
        else:
            return 0
        
       

    def _get_sepsis_onset_hour_or_last_hour(self, x):
        hour_idx = 0
        for hour in x:
            SOFA_score = self._get_sofa_score(57, hour[57]) + self._get_sofa_score(20, hour[20]) + self._get_sofa_score(34, hour[34])
            if SOFA_score >= 2:
                return hour_idx, torch.tensor(1, dtype=torch.float32)
            hour_idx += 1
        return int(x[0][2]*24), torch.tensor(0, dtype=torch.float32) ## Get the first hour's 3rd feature, which is length of stay of this icu
  
    def __getitem__(self, index):
        target_hadm_idx, target_icu_idx = self.available_icu_stays[index]
        x = self.data_tensor[target_hadm_idx, target_icu_idx] 

        l = int(x[0][2]*24) ## Get the first hour's 3rd feature, which is length of stay of this icu
        target_hadm_id = self.hadm_idx_id_dict[target_hadm_idx]
        if self.hadm_id_isSepsis_dict[target_hadm_id] == False:
            y = torch.tensor(0, dtype=torch.float32)
        else:
            l, y = self._get_sepsis_onset_hour_or_last_hour(x)
        return x, y, l

dataset = CustomDataset(final_tensor, hadm_id_isSepsis_dict, hadm_idx_id_dict)

In [ ]:
from torch.utils.data.dataset import random_split

split = int(len(dataset) * 0.8)
lengths = [split, len(dataset) - split]
train_dataset, test_dataset = random_split(dataset, lengths)
print("length of train dataset:", len(train_dataset))
print("length of test dataset:", len(test_dataset))

length of train dataset: 1434
length of test dataset: 359


In [ ]:
def collate_fn(data):
    sequences, labels, lengths = zip(*data)
    x = torch.stack(sequences, dim=0)
    y = torch.tensor(labels, dtype=torch.float)  
    l = torch.tensor(lengths, dtype=torch.long)
    return x, y, l
  
def get_last_visit(hidden_states, length):
    return hidden_states[range(hidden_states.shape[0]), length - 1, :]

In [ ]:

## test
from torch.utils.data import DataLoader
loader = DataLoader(train_dataset, batch_size=2, collate_fn=collate_fn)
loader_iter = iter(loader)
x, y, l = next(loader_iter)

print(x.shape)
print(y.shape)
print(l)


torch.Size([2, 720, 826])
torch.Size([2])
tensor([ 0, 44])


In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

In [ ]:
def get_last_visit(hidden_states, length):
    return hidden_states[range(hidden_states.shape[0]), length - 1, :]

In [ ]:
'''
## TEST 
import random
max_num_visits = 10
batch_size = 16
hidden_dim = 100

hidden_states = torch.randn((batch_size, max_num_visits, hidden_dim))
lengths = torch.tensor([random.randint(1, max_num_visits) for _ in range(batch_size)])
out = get_last_visit(hidden_states, lengths)
'''

'\n## TEST \nimport random\nmax_num_visits = 10\nbatch_size = 16\nhidden_dim = 100\n\nhidden_states = torch.randn((batch_size, max_num_visits, hidden_dim))\nlengths = torch.tensor([random.randint(1, max_num_visits) for _ in range(batch_size)])\nout = get_last_visit(hidden_states, lengths)\n'

In [ ]:


import torch.nn as nn
import torch.nn.functional as F
class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.LSTM(input_size = FEATURE_SIZE, hidden_size = 800, batch_first=True,dtype=torch.float16)
        self.fc = nn.Linear(800, 2)
        self.softmax = nn.Softmax(dim=1)
  
    def forward(self, x, length):
        batch_size = x.shape[0]
        x = x.to(torch.float16)
        output, _ = self.rnn(x)
        true_h_n = get_last_visit(output, length)
        logits = torch.relu(self.fc(true_h_n))
        probs = self.softmax(logits)
        '''
        pred = []
        for res in probs:
          pred.append(res)
        '''
        return probs

model = RNN()
model

RNN(
  (rnn): LSTM(826, 800, batch_first=True)
  (fc): Linear(in_features=800, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
### TPU
accelerator = Accelerator()
device = accelerator.device
model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)

In [ ]:
from sklearn.metrics import *

#input: Y_score,Y_pred,Y_true
#output: accuracy, auc, precision, recall, f1-score
def classification_metrics(Y_score, Y_pred, Y_true):
    auc, precision, recall =   roc_auc_score(Y_true, Y_score), \
                               precision_score(Y_true, Y_pred), \
                                recall_score(Y_true, Y_pred)
    return auc, precision, recall


#input: model, loader
def evaluate(model, loader):
    model.eval()
    all_y_true = torch.LongTensor()
    all_y_pred = torch.LongTensor()
    all_y_score = torch.FloatTensor()
    

    for x, y, l in loader:

        # pass the input through the model
        y_hat = model(x, l)
        y_hat = y_hat.select(dim=1, index=0)
        # convert shape from [batch size, 1] to [batch size]
        y_pred = (y_hat > 0.5).type(torch.float16)
        
        y_hat = torch.nan_to_num(y_hat)

        all_y_true = torch.cat((all_y_true, y.to('cpu')), dim=0)
        all_y_pred = torch.cat((all_y_pred,  y_pred.to('cpu')), dim=0)
        all_y_score = torch.cat((all_y_score,  y_hat.to('cpu')), dim=0)
    
    all_y_true = torch.cat((torch.tensor([1]), all_y_true.to('cpu')), dim=0) ## According to the paper, it up-scaling the septic cases
    all_y_pred = torch.cat((torch.tensor([1]), all_y_pred.to('cpu')), dim=0)
    all_y_score = torch.cat((torch.tensor([1]), all_y_score.to('cpu')), dim=0)

    acc, auc, precision, recall, f1 = classification_metrics(all_y_score.detach().numpy(), 
                                                             all_y_pred.detach().numpy(), 
                                                             all_y_true.detach().numpy())
    print(f"auc: {auc:.3f}, precision: {precision:.3f}, recall: {recall:.3f}")
    return

In [ ]:
#train_init = evaluate(model, train_loader)

In [ ]:
n_epochs = 50

# prep model for training
for epoch in range(n_epochs):
    model.train()
    train_loss = 0
    for x, y, l in train_loader:
        x = torch.nan_to_num(x)
        """ Step 1. clear gradients """
        optimizer.zero_grad()
        """  Step 2. perform forward pass using `model`, save the output to y_hat """
        y_hat = model(x, l)
        """ Step 3. calculate the loss using `criterion`, save the output to loss. """
        
        #y_hat = torch.nan_to_num(y_hat)
        y_hat = torch.select(y_hat, index=0, dim=1)
        
        loss = criterion(y_hat, y)
        """ Step 4. backward pass """
        #loss.backward()
        accelerator.backward(loss)
        """ Step 5. optimization """
        optimizer.step()
        """ Step 6. record loss """
        train_loss += loss.item()
        
    train_loss = train_loss / len(train_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))
    evaluate(model, train_loader)
    evaluate(model, test_loader)